In [80]:
def not_in_new_ride(a, b):
    ans=a
    # リストaからリストbに含まれている要素を取り除く
    for item in b:
        if item in a:
            ans.remove(item)
    return ans


# 24時間表示を分に変換
def calculate_minutes(time_str):
    # 時刻を時間と分に分割
    hours, mins = map(int, time_str.split(":"))
    
    # 9:00 を0分、20:00 を660分として計算
    start_time = 9 * 60  # 9:00を分単位に変換
    end_time = 20 * 60  # 20:00を分単位に変換
    
    # 入力された時刻を分単位に変換
    total_minutes = hours * 60 + mins
    
    # 24時間以上の場合は調整
    if total_minutes < start_time:
        total_minutes += 24 * 60
    
    # 開始時刻からの経過時間を計算
    elapsed_minutes = total_minutes - start_time
    
    return elapsed_minutes


# 分を24時間表示に変換
def calculate_time(minutes):
    # 9:00 を0分、20:00 を660分として計算
    start_time = 9 * 60  # 9:00を分単位に変換
    end_time = 20 * 60  # 20:00を分単位に変換
    
    total_minutes = start_time + minutes
    
    # 時間と分に変換
    hours = total_minutes // 60
    mins = total_minutes % 60
    
    # 24時間表記に変換
    if hours >= 24:
        hours -= 24
    
    # 時間と分を文字列に変換して返す
    return f"{hours:02d}:{mins:02d}"


def ride_printer(date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time):
    print("-" * 40)
    print(f"~ {date} の待ち時間データによる乗車プラン ~")
    print("乗車時間　：　アトラクション名")
    for i in range(len(new_ride_lst)):
        print(f"{calculate_time(ride_time[i])}　： {attractions.get(new_ride_lst[i])}")
    print('トータル所要時間：',total_time)
    print('トータル満足度　：',total_popularity)
    print("-" * 40)
    if not(len(ride_lst)==len(new_ride_lst)):
        print("※全てのアトラクションを乗ることができません")
        print("プランから除外されたアトラクション")
        not_ride = not_in_new_ride(ride_lst, new_ride_lst)
        for i in not_ride:
            print(f"{i}　： {attractions.get(i)}")

In [93]:
import ast
from collections import Counter

# レビュー評価（popularity）
popularity = [476, 465, 473, 472, 480, 452, 461, 478]

def find_optimal_plan(date, attractions):
    # 入力のアトラクションの出現回数をカウント
    attraction_counts = Counter(attractions)
    
    # 日付でフィルタリング
    date_filtered_df = df[df['日付'] == date]
    
    # 指定アトラクションの出現回数が一致する行を抽出
    def matches_plan(optimal_plan):
        plan_counts = Counter(optimal_plan)
        return all(plan_counts.get(attraction, 0) >= count for attraction, count in attraction_counts.items())

    filtered_df = date_filtered_df[date_filtered_df['最適プラン'].apply(matches_plan)]
    
    if filtered_df.empty:
        print("指定された条件に合うプランは見つかりませんでした。")
        
        # '日付'が'median'のデータを参照
        median_df = df[df['日付'] == 'median']
        
        # 'median'のデータが存在する場合、トータル満足度が最大のプランを取得
        if not median_df.empty:
            max_satisfaction_plan = median_df.loc[median_df['トータル満足度'].idxmax()]
            print(f"参考データ（インデックス:{max_satisfaction_plan.name}, 日付:{max_satisfaction_plan['日付']}）")
        else:
            print("日付が'median'のデータは存在しません。すべてのデータから最適プランを選択します。")
            # 'median'がない場合は、全データから最適プランを選択
            max_satisfaction_plan = df.loc[df['トータル満足度'].idxmax()]
        
        # 足枷となっているアトラクションを除外して再検索
        excluded_attractions = []
        for attraction in attraction_counts:
            # 条件に合わないアトラクションを除外
            date_filtered_df_without_attraction = date_filtered_df[date_filtered_df['最適プラン'].apply(lambda x: attraction not in x)]
            if not date_filtered_df_without_attraction.empty:
                excluded_attractions.append(attraction)
                filtered_df = date_filtered_df_without_attraction[date_filtered_df_without_attraction['最適プラン'].apply(matches_plan)]
        
        if excluded_attractions:
            print(f"除外したアトラクション: {excluded_attractions}")
            if not filtered_df.empty:
                max_satisfaction_plan = filtered_df.loc[filtered_df['トータル満足度'].idxmax()]
            else:
                print("除外後も条件に合うプランは見つかりませんでした。")
        
    else:
        # トータル満足度が最大のプランを取得
        max_satisfaction_plan = filtered_df.loc[filtered_df['トータル満足度'].idxmax()]
    
    # 結果を出力
    print(f"参考データ：{max_satisfaction_plan['日付']}")
    print("最適プラン:", max_satisfaction_plan['最適プラン'])
    print("タイムスケジュール:", max_satisfaction_plan['タイムスケジュール'])
    print("トータル満足度:", max_satisfaction_plan['トータル満足度'])
    print("トータル所要時間:", max_satisfaction_plan['トータル所要時間'])
    print()
    
    # ここでride_printer関数を呼び出す
    ride_lst = max_satisfaction_plan['最適プラン']
    new_ride_lst = max_satisfaction_plan['最適プラン']
    
    # タイムスケジュールがリストとして渡されるように修正
    ride_time = max_satisfaction_plan['タイムスケジュール']
    
    # もしタイムスケジュールが文字列であれば、リストに変換
    if isinstance(ride_time, str):
        ride_time = ast.literal_eval(ride_time)  # 文字列をリストに変換
    
    # ride_timeがリスト形式であることを確認
    if not isinstance(ride_time, list):
        print("タイムスケジュールがリスト形式でありません。")
        return
    
    # 同じアトラクションが連続している場合、そのアトラクションを削除
    filtered_ride_lst = []
    filtered_ride_time = []
    
    for i in range(len(ride_lst)):
        if i == 0 or ride_lst[i] != ride_lst[i - 1]:
            filtered_ride_lst.append(ride_lst[i])
            filtered_ride_time.append(ride_time[i])

    # 結果の表示
    print("更新された最適プラン:", filtered_ride_lst)
    print("更新されたタイムスケジュール:", filtered_ride_time)
    
    # トータル満足度を再計算
    # filtered_ride_lst に基づいて、対応する popularity を取得
    total_popularity = sum([popularity[ride] for ride in filtered_ride_lst])
    
    # 再計算されたトータル満足度
    total_satisfaction = total_popularity
    
    print("再計算されたトータル満足度:", total_satisfaction)
    
    # 変数を準備してride_printer関数に渡す
    total_time = max_satisfaction_plan['トータル所要時間']
    
    ride_printer(max_satisfaction_plan['日付'], total_satisfaction, total_time, ride_lst, filtered_ride_lst, filtered_ride_time)

# 使用例
date_input = '2022-08-01'  # 予測したい日付
attractions_input = [0,1,2,3,4]  # 乗りたいアトラクション番号リスト
find_optimal_plan(date_input, attractions_input)


参考データ：2022-08-01
最適プラン: [6, 4, 0, 5, 4, 6, 2, 3, 4, 2, 1, 4, 3, 4, 0]
タイムスケジュール: [0, 45, 90, 180, 210, 240, 300, 360, 405, 435, 495, 570, 600, 645, 660]
トータル満足度: 7081.0
トータル所要時間: 675.0

更新された最適プラン: [6, 4, 0, 5, 4, 6, 2, 3, 4, 2, 1, 4, 3, 4, 0]
更新されたタイムスケジュール: [0, 45, 90, 180, 210, 240, 300, 360, 405, 435, 495, 570, 600, 645, 660]
再計算されたトータル満足度: 7081
----------------------------------------
~ 2022-08-01 の待ち時間データによる乗車プラン ~
乗車時間　：　アトラクション名
09:00　： マジックランプシアター
09:45　： インディージョーンズ
10:30　： ソアリン
12:00　： レイジングスピリッツ
12:30　： インディージョーンズ
13:00　： マジックランプシアター
14:00　： タワーオブテラー
15:00　： センターオブジアース
15:45　： インディージョーンズ
16:15　： タワーオブテラー
17:15　： トイストーリーマニア
18:30　： インディージョーンズ
19:00　： センターオブジアース
19:45　： インディージョーンズ
20:00　： ソアリン
トータル所要時間： 675.0
トータル満足度　： 7081
----------------------------------------
